In [0]:
!pip install Keras==2.2.4

In [0]:
!pip uninstall -y tensorflow

In [0]:
!pip install tensorflow-gpu==1.14.0 --ignore-installed

  Using cached https://files.pythonhosted.org/packages/76/04/43153bfdfcf6c9a4c38ecdb971ca9a75b9a791bb69a764d652c359aca504/tensorflow_gpu-1.14.0-cp36-cp36m-manylinux1_x86_64.whl
     |████████████████████████████████| 20.2MB 75.6MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 1.3MB 45.4MB/s 
     |████████████████████████████████| 112kB 57.0MB/s 
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl
     |████████████████████████████████| 61kB 9.9MB/s 
  Using cached https://files.pythonhosted.org/packages/91/2d/2ed263449a078cd9c8a9ba50ebd50123adf1f8cfbea1492f9084169b89d9/tensorboard-1.14.0-py3-none-any.whl
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 2.9MB 52.6MB/s 
     |████████████████████████████████| 583kB 55.8MB/s 
     |████████████████████████████████| 92kB 13.2MB

In [0]:
import pandas as pd
import numpy as np

from numpy.random import seed
from tensorflow import set_random_seed
import tensorflow as tf
import random as rn
import os
from nltk import tokenize 
import nltk
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#2. Get the file
if os.name=='posix':
    
    # GOOGLE COLAB SETUP
    # Load the Drive helper and mount
    from google.colab import drive
    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
    codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/AOBDL_DL'
    cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models_facebook'
    models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models_facebook'
    
else:
    data_path          = '../data'
    codes_path         = '../AOBDL_DL'
    cv_models_path     = '../cv_models_facebook'
    models_path        = '../models_facebook'

In [0]:
import sys
sys.path.append(f'{codes_path}')

In [0]:
if tf.test.is_gpu_available():
  import models
else:
  import models_no_gpu as models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/facebook_cleaned_no_punkt.csv') 
train.drop(['id', 'agr'], axis=1, inplace=True)
train.text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42      
X, X_test, y, y_test  = train_test_split(train.text, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )

X = np.array(X)
y = np.array(y)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
max_features = 7000
maxlen       = 600
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150
threshold    = 0.4


#max_sen_len    = 100
#max_sent_amount = 4

seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [0]:
length = pd.Series(X).apply(lambda x: len(x.split()))

In [0]:
length_sen = pd.Series(X).apply(lambda x: len(x.split('.|!|?')))

In [0]:
length.describe()

count    11300.000000
mean        20.415929
std         27.530828
min          1.000000
25%          9.000000
50%         13.000000
75%         23.000000
max        740.000000
dtype: float64

# LSTM

In [0]:
models.train_model(
    X=X, 
    y=y,
    mtype = 'LSTM',
    cv=True, 
    nfolds=5, 
    epochs=4, 
    cv_models_path=cv_models_path, 
    train=False, 
    rs=rs, 
    max_features=max_features, 
    maxlen=maxlen,
    threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
9040/9040 [==============================] - 6s 671us/step - loss: 0.6786 - acc: 0.5912
Epoch 2/4
9040/9040 [==============================] - 3s 310us/step - loss: 0.5696 - acc: 0.7144
Epoch 3/4
9040/9040 [==============================] - 3s 311us/step - loss: 0.5039 - acc: 0.7657
Epoch 4/4
2260/2260 [==============================] - 2s 675us/step
 0.4 fold 0 precision 0.698 recall 0.837 fscore 0.761
fold 0 average precision 0.786
fold 0 roc auc 0.76
 fold 1
Fitting
Epoch 1/4
9040/9040 [==============================] - 3s 343us/step - loss: 0.6690 - acc: 0.5917
Epoch 2/4
9040/9040 [==============================] - 3s 308us/step - loss: 0.6830 - acc: 0.6926
Epoch 3/4
9040/9040 [==============================] - 3s 308us/step - loss: 0.5035 - acc: 0.7551
Epoch 4/4
2260/2260 [==============================] - 0s 118us/step
 0.4 fold 1 precision 0.691 recall 0.851 fscore 0.763
fold 1 average precision 0.798
fold 1 roc auc 0.77
 fold 2
Fitting
Epoch 1/4
9040/9

In [0]:
models.train_model(X=X, 
                    y=y, 
                    X_test=X_test, 
                    y_test=y_test, 
                    mtype = 'LSTM', 
                    cv=False, 
                    epochs=4, 
                    cv_models_path=models_path, 
                    train=False, 
                    rs=rs,
                    max_features=max_features, 
                    maxlen=maxlen,
                    threshold=threshold)

Fitting
Epoch 1/4
11300/11300 [==============================] - 4s 342us/step - loss: 0.6801 - acc: 0.6034
Epoch 2/4
11300/11300 [==============================] - 3s 306us/step - loss: 0.5848 - acc: 0.7166
Epoch 3/4
11300/11300 [==============================] - 3s 304us/step - loss: 0.5034 - acc: 0.7650
Epoch 4/4
4616/4616 [==============================] - 1s 109us/step
_________________________________
PR-C is 0.806
_________________________________

_________________________________
ROC AUC is 0.784
_________________________________
_________________________________
FScore is 0.773
_________________________________



# GRU

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'GRU', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
9040/9040 [==============================] - 4s 433us/step - loss: 0.6653 - acc: 0.5939
Epoch 2/4
9040/9040 [==============================] - 3s 379us/step - loss: 0.5558 - acc: 0.7244
Epoch 3/4
9040/9040 [==============================] - 3s 379us/step - loss: 0.4713 - acc: 0.7738
Epoch 4/4
2260/2260 [==============================] - 0s 136us/step
 0.4 fold 0 precision 0.693 recall 0.882 fscore 0.776
fold 0 average precision 0.797
fold 0 roc auc 0.77
 fold 1
Fitting
Epoch 1/4
9040/9040 [==============================] - 4s 417us/step - loss: 0.6656 - acc: 0.5930
Epoch 2/4
9040/9040 [==============================] - 3s 380us/step - loss: 0.5541 - acc: 0.7279
Epoch 3/4
9040/9040 [==============================] - 3s 378us/step - loss: 0.4854 - acc: 0.7691
Epoch 4/4
2260/2260 [==============================] - 0s 108us/step
 0.4 fold 1 precision 0.682 recall 0.886 fscore 0.77
fold 1 average precision 0.81
fold 1 roc auc 0.779
 fold 2
Fitting
Epoch 1/4
9040/90

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'GRU', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
11300/11300 [==============================] - 5s 410us/step - loss: 0.6550 - acc: 0.6093
Epoch 2/4
11300/11300 [==============================] - 4s 384us/step - loss: 0.5414 - acc: 0.7348
Epoch 3/4
11300/11300 [==============================] - 4s 382us/step - loss: 0.4736 - acc: 0.7717
Epoch 4/4
4616/4616 [==============================] - 0s 98us/step
_________________________________
PR-C is 0.8
_________________________________

_________________________________
ROC AUC is 0.776
_________________________________
_________________________________
FScore is 0.774
_________________________________



# CNN

In [0]:
cnn_threshold = 0.3

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'CNN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=cnn_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
2260/2260 [==============================] - 0s 67us/step
 0.3 fold 0 precision 0.685 recall 0.888 fscore 0.774
fold 0 average precision 0.81
fold 0 roc auc 0.783
 fold 1
Fitting
2260/2260 [==============================] - 0s 43us/step
 0.3 fold 1 precision 0.691 recall 0.871 fscore 0.771
fold 1 average precision 0.814
fold 1 roc auc 0.785
 fold 2
Fitting
2260/2260 [==============================] - 0s 41us/step
 0.3 fold 2 precision 0.675 recall 0.895 fscore 0.77
fold 2 average precision 0.804
fold 2 roc auc 0.775
 fold 3
Fitting
2260/2260 [==============================] - 0s 43us/step
 0.3 fold 3 precision 0.669 recall 0.936 fscore 0.78
fold 3 average precision 0.809
fold 3 roc auc 0.787
 fold 4
Fitting
2260/2260 [==============================] - 0s 43us/step
 0.3 fold 4 precision 0.695 recall 0.896 fscore 0.783
fold 4 average precision 0.807
fold 4 roc auc 0.783
PR-C 0.809
ROC AUC 0.783
FScore 0.775
PR-C std 0.003
ROC AUC std 0.004
FScore std 0.005


In [0]:
models.train_model(X=X, 
                  y=y, 
                  X_test=X_test, 
                  y_test=y_test, 
                  mtype = 'CNN', 
                  cv=False, 
                  epochs=4, 
                  cv_models_path=models_path, 
                  train=False, 
                  rs=rs,
                  max_features=max_features, 
                  maxlen=maxlen,
                  threshold=cnn_threshold)

Fitting
4616/4616 [==============================] - 0s 37us/step
_________________________________
PR-C is 0.818
_________________________________

_________________________________
ROC AUC is 0.788
_________________________________
_________________________________
FScore is 0.775
_________________________________



# BGRU

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
9040/9040 [==============================] - 7s 756us/step - loss: 0.6674 - acc: 0.5950
Epoch 2/4
9040/9040 [==============================] - 6s 697us/step - loss: 0.5623 - acc: 0.7253
Epoch 3/4
9040/9040 [==============================] - 6s 694us/step - loss: 0.4907 - acc: 0.7674
Epoch 4/4
2260/2260 [==============================] - 0s 212us/step
 0.4 fold 0 precision 0.701 recall 0.862 fscore 0.773
fold 0 average precision 0.803
fold 0 roc auc 0.775
 fold 1
Fitting
Epoch 1/4
9040/9040 [==============================] - 7s 746us/step - loss: 0.6721 - acc: 0.5850
Epoch 2/4
9040/9040 [==============================] - 6s 696us/step - loss: 0.5696 - acc: 0.7209
Epoch 3/4
9040/9040 [==============================] - 6s 696us/step - loss: 0.4887 - acc: 0.7676
Epoch 4/4
2260/2260 [==============================] - 0s 205us/step
 0.4 fold 1 precision 0.672 recall 0.905 fscore 0.772
fold 1 average precision 0.813
fold 1 roc auc 0.78
 fold 2
Fitting
Epoch 1/4
9040/

In [0]:
models.train_model(X=X, 
                y=y, 
                X_test=X_test, 
                y_test=y_test, 
                mtype = 'BGRU', 
                cv=False, 
                epochs=4, 
                cv_models_path=models_path, 
                train=False,
                rs=rs,
                max_features=max_features, 
                maxlen=maxlen,
                threshold=threshold)

Fitting
4616/4616 [==============================] - 1s 269us/step
_________________________________
PR-C is 0.801
_________________________________

_________________________________
ROC AUC is 0.773
_________________________________
_________________________________
FScore is 0.771
_________________________________



# BLSTM

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BLSTM', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
9040/9040 [==============================] - 7s 726us/step - loss: 0.6782 - acc: 0.5753
Epoch 2/4
9040/9040 [==============================] - 5s 563us/step - loss: 0.6125 - acc: 0.6809
Epoch 3/4
9040/9040 [==============================] - 5s 566us/step - loss: 0.5236 - acc: 0.7510
Epoch 4/4
2260/2260 [==============================] - 1s 411us/step
 0.4 fold 0 precision 0.701 recall 0.862 fscore 0.773
fold 0 average precision 0.785
fold 0 roc auc 0.775
 fold 1
Fitting
Epoch 1/4
9040/9040 [==============================] - 6s 612us/step - loss: 0.6847 - acc: 0.5696
Epoch 2/4
9040/9040 [==============================] - 5s 562us/step - loss: 0.6025 - acc: 0.6861
Epoch 3/4
9040/9040 [==============================] - 5s 561us/step - loss: 0.5212 - acc: 0.7539
Epoch 4/4
2260/2260 [==============================] - 1s 227us/step
 0.4 fold 1 precision 0.675 recall 0.907 fscore 0.774
fold 1 average precision 0.8
fold 1 roc auc 0.776
 fold 2
Fitting
Epoch 1/4
9040/9

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BLSTM', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
11300/11300 [==============================] - 7s 599us/step - loss: 0.6676 - acc: 0.5851
Epoch 2/4
11300/11300 [==============================] - 6s 559us/step - loss: 0.5768 - acc: 0.7129
Epoch 3/4
11300/11300 [==============================] - 6s 559us/step - loss: 0.5509 - acc: 0.7575
Epoch 4/4
4616/4616 [==============================] - 1s 208us/step
_________________________________
PR-C is 0.802
_________________________________

_________________________________
ROC AUC is 0.78
_________________________________
_________________________________
FScore is 0.775
_________________________________



# BGRU + Max Pooling

In [0]:
BGRU_max_threshold = 0.2

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_max', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=BGRU_max_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
2260/2260 [==============================] - 1s 358us/step
 0.2 fold 0 precision 0.638 recall 0.969 fscore 0.769
fold 0 average precision 0.81
fold 0 roc auc 0.784
 fold 1
Fitting
2260/2260 [==============================] - 0s 208us/step
 0.2 fold 1 precision 0.636 recall 0.973 fscore 0.769
fold 1 average precision 0.814
fold 1 roc auc 0.784
 fold 2
Fitting
2260/2260 [==============================] - 0s 210us/step
 0.2 fold 2 precision 0.645 recall 0.94 fscore 0.765
fold 2 average precision 0.8
fold 2 roc auc 0.77
 fold 3
Fitting
2260/2260 [==============================] - 0s 209us/step
 0.2 fold 3 precision 0.665 recall 0.929 fscore 0.775
fold 3 average precision 0.809
fold 3 roc auc 0.783
 fold 4
Fitting
2260/2260 [==============================] - 0s 211us/step
 0.2 fold 4 precision 0.67 recall 0.931 fscore 0.779
fold 4 average precision 0.815
fold 4 roc auc 0.79
PR-C 0.81
ROC AUC 0.782
FScore 0.771
PR-C std 0.005
ROC AUC std 0.007
FScore std 0.005


In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_max', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=BGRU_max_threshold)

Fitting
4616/4616 [==============================] - 1s 200us/step
_________________________________
PR-C is 0.816
_________________________________

_________________________________
ROC AUC is 0.788
_________________________________
_________________________________
FScore is 0.779
_________________________________



# BGRU + attention

In [0]:
BGRU_att_len = 50
BGRU_att_threshold = 0.1

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_att', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=BGRU_att_len,
                   threshold=BGRU_att_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
2260/2260 [==============================] - 0s 102us/step
 0.1 fold 0 precision 0.615 recall 0.984 fscore 0.757
fold 0 average precision 0.816
fold 0 roc auc 0.786
 fold 1
Fitting
2260/2260 [==============================] - 0s 56us/step
 0.1 fold 1 precision 0.605 recall 0.997 fscore 0.753
fold 1 average precision 0.816
fold 1 roc auc 0.787
 fold 2
Fitting
2260/2260 [==============================] - 0s 54us/step
 0.1 fold 2 precision 0.604 recall 0.982 fscore 0.748
fold 2 average precision 0.803
fold 2 roc auc 0.772
 fold 3
Fitting
2260/2260 [==============================] - 0s 57us/step
 0.1 fold 3 precision 0.612 recall 0.983 fscore 0.755
fold 3 average precision 0.809
fold 3 roc auc 0.782
 fold 4
Fitting
2260/2260 [==============================] - 0s 63us/step
 0.1 fold 4 precision 0.626 recall 0.982 fscore 0.764
fold 4 average precision 0.814
fold 4 roc auc 0.788
PR-C 0.811
ROC AUC 0.783
FScore 0.755
PR-C std 0.005
ROC AUC std 0.006
FScore std 0.005


In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_att', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=BGRU_att_len,
                   threshold=BGRU_att_threshold)

Fitting
4616/4616 [==============================] - 0s 38us/step
_________________________________
PR-C is 0.807
_________________________________

_________________________________
ROC AUC is 0.776
_________________________________
_________________________________
FScore is 0.77
_________________________________



# BGRU + Average Pooling

In [0]:
BGRU_avg_len = 50
BGRU_avg_threshold = 0.3

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_avg', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False,
                   rs=rs,
                   max_features=max_features, 
                   maxlen=BGRU_avg_len,
                   threshold=BGRU_avg_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
9040/9040 [==============================] - 3s 281us/step - loss: 0.6799 - acc: 0.5705
Epoch 2/4
9040/9040 [==============================] - 1s 79us/step - loss: 0.6222 - acc: 0.6659
Epoch 3/4
9040/9040 [==============================] - 1s 79us/step - loss: 0.5111 - acc: 0.7553
Epoch 4/4
2260/2260 [==============================] - 1s 331us/step
 0.3 fold 0 precision 0.654 recall 0.957 fscore 0.777
fold 0 average precision 0.811
fold 0 roc auc 0.783
 fold 1
Fitting
Epoch 1/4
9040/9040 [==============================] - 1s 125us/step - loss: 0.6782 - acc: 0.5733
Epoch 2/4
9040/9040 [==============================] - 1s 78us/step - loss: 0.5961 - acc: 0.6830
Epoch 3/4
9040/9040 [==============================] - 1s 78us/step - loss: 0.5123 - acc: 0.7605
Epoch 4/4
2260/2260 [==============================] - 0s 60us/step
 0.3 fold 1 precision 0.674 recall 0.926 fscore 0.78
fold 1 average precision 0.812
fold 1 roc auc 0.783
 fold 2
Fitting
Epoch 1/4
9040/9040 

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_avg', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=BGRU_avg_len,
                   threshold=BGRU_avg_threshold)

Fitting
4616/4616 [==============================] - 0s 33us/step
_________________________________
PR-C is 0.806
_________________________________

_________________________________
ROC AUC is 0.783
_________________________________
_________________________________
FScore is 0.778
_________________________________



# psHAN

In [0]:
psHAN_max_sen_len = 20
psHAN_max_sent_amount = 8
psHAN_threshold = 0.2

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'psHAN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=psHAN_max_sent_amount*psHAN_max_sen_len,
                   max_sen_len=psHAN_max_sen_len,
                   max_sent_amount=psHAN_max_sent_amount,
                   threshold=psHAN_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
2260/2260 [==============================] - 0s 140us/step
 0.2 fold 0 precision 0.648 recall 0.961 fscore 0.774
fold 0 average precision 0.805
fold 0 roc auc 0.773
 fold 1
Fitting
2260/2260 [==============================] - 0s 105us/step
 0.2 fold 1 precision 0.665 recall 0.919 fscore 0.771
fold 1 average precision 0.797
fold 1 roc auc 0.771
 fold 2
Fitting
2260/2260 [==============================] - 0s 101us/step
 0.2 fold 2 precision 0.67 recall 0.912 fscore 0.772
fold 2 average precision 0.802
fold 2 roc auc 0.774
 fold 3
Fitting
2260/2260 [==============================] - 0s 99us/step
 0.2 fold 3 precision 0.645 recall 0.961 fscore 0.772
fold 3 average precision 0.8
fold 3 roc auc 0.776
 fold 4
Fitting
2260/2260 [==============================] - 0s 103us/step
 0.2 fold 4 precision 0.697 recall 0.901 fscore 0.786
fold 4 average precision 0.819
fold 4 roc auc 0.796
PR-C 0.805
ROC AUC 0.778
FScore 0.775
PR-C std 0.008
ROC AUC std 0.009
FScore std 0.006


In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'psHAN', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=psHAN_max_sent_amount*psHAN_max_sen_len,
                   max_sen_len=psHAN_max_sen_len,
                   max_sent_amount=psHAN_max_sent_amount,
                   threshold=psHAN_threshold)

Fitting
4616/4616 [==============================] - 0s 76us/step
_________________________________
PR-C is 0.809
_________________________________

_________________________________
ROC AUC is 0.782
_________________________________
_________________________________
FScore is 0.778
_________________________________



# HAN

In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/facebook_cleaned_sent_punkt.csv') 
train.drop(['id', 'agr'], axis=1, inplace=True)
train.text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42      
X, X_test, y, y_test  = train_test_split(train.text, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )

X = np.array(X)
y = np.array(y)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
HAN_max_sen_len = 20
HAN_max_sent_amount = 8
HAN_threshold = 0.3

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'HAN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=HAN_max_sen_len*HAN_max_sent_amount,
                   max_sen_len=HAN_max_sen_len,
                   max_sent_amount=HAN_max_sent_amount,
                   threshold=HAN_threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
0
0
Fitting
2260/2260 [==============================] - 0s 135us/step
 0.3 fold 0 precision 0.68 recall 0.866 fscore 0.762
fold 0 average precision 0.79
fold 0 roc auc 0.76
 fold 1
0
0
Fitting
2260/2260 [==============================] - 0s 96us/step
 0.3 fold 1 precision 0.656 recall 0.944 fscore 0.774
fold 1 average precision 0.799
fold 1 roc auc 0.768
 fold 2
0
0
Fitting
2260/2260 [==============================] - 0s 99us/step
 0.3 fold 2 precision 0.645 recall 0.952 fscore 0.769
fold 2 average precision 0.788
fold 2 roc auc 0.763
 fold 3
0
0
Fitting
2260/2260 [==============================] - 0s 101us/step
 0.3 fold 3 precision 0.682 recall 0.881 fscore 0.769
fold 3 average precision 0.79
fold 3 roc auc 0.773
 fold 4
0
0
Fitting
2260/2260 [==============================] - 0s 106us/step
 0.3 fold 4 precision 0.696 recall 0.869 fscore 0.773
fold 4 average precision 0.778
fold 4 roc auc 0.76
PR-C 0.789
ROC AUC 0.765
FScore 0.769
PR-C std 0.007
ROC AUC std 0.005
FScore std 

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'HAN', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=HAN_max_sen_len*HAN_max_sent_amount,
                   max_sen_len=HAN_max_sen_len,
                   max_sent_amount=HAN_max_sent_amount,
                   threshold=HAN_threshold)

0
10000
0
Fitting
4616/4616 [==============================] - 0s 75us/step
_________________________________
PR-C is 0.795
_________________________________

_________________________________
ROC AUC is 0.768
_________________________________
_________________________________
FScore is 0.767
_________________________________

